## Plotly

* Мы создадим анимационный график с использованием plotly.

#### 1. Подключаемся к базе

In [33]:
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.graph_objects as go

conn = sqlite3.connect('../data/checking-logs.sqlite')

#### 2. Загрузка данных 

In [34]:
query = """
SELECT uid, timestamp
FROM checker;
"""
df = pd.read_sql(query, conn)

df["timestamp"] = pd.to_datetime(df["timestamp"])

df.head()

,uid,timestamp
0,admin_1,2020-04-16 21:12:50.740474
1,admin_1,2020-04-16 21:12:54.708365
2,admin_1,2020-04-16 21:46:47.769088
3,admin_1,2020-04-16 21:46:48.121217
4,admin_1,2020-04-16 21:53:01.862637


#### 3. Обработка данных

In [35]:
df = df.dropna(subset=['uid'])

df = df[df["uid"].str.startswith("user")]

df["date"] = df["timestamp"].dt.date

df_grouped = df.groupby(["date", "uid"]).size().reset_index(name="commits")

df_grouped["cumulative_commits"] = df_grouped.groupby("uid")["commits"].cumsum()

df_grouped.head()

,date,uid,commits,cumulative_commits
0,2020-04-17,user_4,23,23
1,2020-04-18,user_12,7,7
2,2020-04-18,user_14,42,42
3,2020-04-18,user_17,1,1
4,2020-04-18,user_2,16,16


#### 4. Построение графика

In [36]:
unique_dates = sorted(df_grouped["date"].unique())

fig = go.Figure()

users = df_grouped["uid"].unique()
for user in users:
    user_data = df_grouped[df_grouped["uid"] == user]
    fig.add_trace(go.Scatter(x=user_data["date"], y=user_data["cumulative_commits"],
                             mode='lines', name=user,
                             line=dict(width=2)))

frames = []
for date in unique_dates:
    frame_data = []
    for user in users:
        user_data = df_grouped[(df_grouped["uid"] == user) & (df_grouped["date"] <= date)]
        frame_data.append(go.Scatter(x=user_data["date"], y=user_data["cumulative_commits"],
                                     mode='lines+markers', name=user,
                                     line=dict(width=2), marker=dict(size=6)))
    frames.append(go.Frame(data=frame_data, name=str(date)))

fig.frames = frames

fig.update_layout(
    updatemenus=[{
        "buttons": [
            {"args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}],
             "label": "Play", "method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}],
             "label": "Pause", "method": "animate"}
        ],
        "direction": "left", "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }]
)

fig.update_layout(title="Динамика коммитов по пользователям", xaxis_title="Дата",
                  yaxis_title="Кумулятивные коммиты", template="plotly_dark", paper_bgcolor="#2E2E2E")

fig.show()
